In [27]:
from pydantic import BaseModel, HttpUrl

class Config(BaseModel):
    url: str
    lang: str

In [28]:
import requests
from langchain.tools import StructuredTool
from bs4 import BeautifulSoup
from langchain_core.messages import HumanMessage, AnyMessage, AIMessage, SystemMessage, ToolMessage

def text_fetcher(url: str) -> str:
    """
    Fetches text from the given URL.
    """
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        clean_text = soup.get_text()
        return clean_text

    else:
        raise Exception(f"Failed to fetch text: {response.status_code}")
        
def summarizer(text:str, model) -> str:
    prompt=f'''
        You are a summarization model. I will provide you with a text, and your task is to summarize the main points concisely. While summarizing, please ensure to:

        1. Focus only on the core information, such as key arguments, facts, and findings.
        2. Ignore any irrelevant content such as advertisements, navigation menus, and any promotional material.
        3. Skip over repetitive or unrelated content that does not contribute to the paper's or article's primary purpose.
        4. Provide a clear, coherent summary in your response.

        Here is the text you need to summarize:

        {text}
    '''.strip()
    response = model.invoke([HumanMessage(content=prompt)])
    return response.content

def translater(text:str, lang:str, model) -> str:
    prompt=f'''
        You are a translation model. I will provide you with a text, and your task is to translate it into {lang}. While translating, please ensure to:

        1. Maintain the original meaning and context of the text.
        2. Use appropriate terminology and phrasing for the target language.
        3. Ensure grammatical correctness in the translated text.
        4. If the text is already in the target language, simply return it as is.
        5. Avoid adding any additional commentary or explanations.

        Here is the text you need to translate:

        {text}
    '''.strip()
    response = model.invoke([HumanMessage(content=prompt)])
    return response.content

def tool_fn(configuration:Config,model) -> str:
    """
    Main function to fetch, summarize, and translate text from a given URL.
    """
    
    text = text_fetcher(configuration.url)
    summary = summarizer(text, model)
    translated=translater(summary, configuration.lang, model)
    
    return translated
    


In [29]:
from langchain_groq import ChatGroq
model=ChatGroq(model='llama3-8b-8192')


config1=Config(
    url="https://zapier.com/blog/deepseek-vs-chatgpt/",
    lang="Hindi",
)

In [30]:
tool = StructuredTool.from_function(
    func=tool_fn,
    name="Research Summarizer",
    description="Fetches text from a URL, summarizes it, and translates it into the specified language."
)

In [31]:
result = tool.invoke({"configuration": config1.model_dump(),"model": model})
result

'यह प्रतिबिंबित करता है कि डीपी सीक और चैटजीपीटी दो एआई टूल हैं, जिनके बीच सबसे अच्छा क्या है, इसका निर्धारण करने के लिए। निम्नांकित संक्षिप्त सारांश है:\n\n* डीपी सीक और चैटजीपीटी दोनों मानवीय प्रणाली (एआई) टूल हैं, जो कोडिंग, चैटबॉट निर्माण और ऑटोमेशन जैसे विभिन्न कार्य कर सकते हैं।\n* चैटजीपीटी ओपनएआई द्वारा विकसित एआई टूल है, जबकि डीपी सीक का उल्लेख नहीं है।\n* लेख का सुझाव है कि चैटजीपीटी एक अधिक संपूर्ण एआई टूल है, जिसमें नेचर लैंग्वेज प्रोसेसिंग, मशीन लर्निंग और ऑटोमेशन जैसे व्यापक सुविधाएं और क्षमताएं हैं।\n* डीपी सीक को इन सुविधाओं और क्षमताओं के साथ उल्लेख नहीं है, लेकिन सुझाव है कि यह कुछ निश्चित कार्यों या उद्योगों के लिए अधिक उपयुक्त हो सकता है।\n* लेख प्रत्यक्ष रूप से किस एआई टूल का सबसे अच्छा क्या है, इसका निर्धारण नहीं करता है, क्योंकि यह चैटजीपीटी की सुविधाओं और क्षमताओं के बारे में अधिक से अधिक फोकस करता है।'